# 路徑與檔案系統 | Paths and File Systems

## 📝 詳解範例 | Worked Examples

---

## 💡 本檔案目的

本檔案提供 **5 個循序漸進的詳解範例**，每個範例包含：
1. **問題描述**：實際應用情境
2. **分析思路**：如何拆解問題
3. **逐步實作**：程式碼 + 註解
4. **執行結果**：預期輸出
5. **知識點總結**：學到什麼

---

## 範例 1：檔案搜尋工具

### 📋 問題描述
建立一個檔案搜尋工具，能夠：
1. 在指定目錄中搜尋特定副檔名的檔案
2. 支援遞迴搜尋子目錄
3. 顯示檔案大小與修改時間
4. 依大小排序結果

**難度**：基礎-中級

### 🔍 分析思路
1. 使用 `rglob()` 遞迴搜尋檔案
2. 使用 `stat()` 取得檔案屬性
3. 將結果存入列表並排序
4. 格式化輸出（檔名、大小、時間）

### 💻 逐步實作

In [ ]:
from pathlib import Path
from datetime import datetime

def search_files(directory, pattern='*', sort_by='size', reverse=True):
    """
    搜尋檔案並顯示詳細資訊
    
    參數:
        directory: 搜尋目錄
        pattern: glob 模式 (預設 '*' 搜尋所有)
        sort_by: 排序方式 ('size', 'name', 'time')
        reverse: 是否反向排序
    """
    # 步驟 1: 驗證目錄
    search_dir = Path(directory)
    if not search_dir.exists() or not search_dir.is_dir():
        print(f"錯誤：{directory} 不是有效的目錄")
        return
    
    print(f"搜尋目錄: {search_dir.resolve()}")
    print(f"搜尋模式: {pattern}")
    print("=" * 80)
    
    # 步驟 2: 搜尋檔案
    files = []
    for file in search_dir.rglob(pattern):
        if file.is_file():  # 只處理檔案
            stat = file.stat()
            files.append({
                'path': file,
                'name': file.name,
                'size': stat.st_size,
                'mtime': stat.st_mtime,
                'relative': file.relative_to(search_dir)
            })
    
    # 步驟 3: 排序
    if sort_by == 'size':
        files.sort(key=lambda x: x['size'], reverse=reverse)
    elif sort_by == 'name':
        files.sort(key=lambda x: x['name'].lower(), reverse=reverse)
    elif sort_by == 'time':
        files.sort(key=lambda x: x['mtime'], reverse=reverse)
    
    # 步驟 4: 顯示結果
    if not files:
        print("未找到任何檔案")
        return
    
    print(f"找到 {len(files)} 個檔案：\n")
    
    # 表頭
    print(f"{'檔名':<30} {'大小':>12} {'修改時間':>20} {'路徑'}")
    print("-" * 80)
    
    # 檔案清單
    for file in files:
        # 格式化大小
        size = file['size']
        if size < 1024:
            size_str = f"{size} B"
        elif size < 1024 * 1024:
            size_str = f"{size / 1024:.1f} KB"
        else:
            size_str = f"{size / (1024 * 1024):.1f} MB"
        
        # 格式化時間
        mtime_str = datetime.fromtimestamp(file['mtime']).strftime('%Y-%m-%d %H:%M')
        
        # 輸出
        name = file['name'][:28] + '..' if len(file['name']) > 30 else file['name']
        print(f"{name:<30} {size_str:>12} {mtime_str:>20} {file['relative']}")
    
    # 統計資訊
    total_size = sum(f['size'] for f in files)
    print("\n" + "=" * 80)
    print(f"總計: {len(files)} 個檔案，總大小: {total_size / 1024:.1f} KB")

# 測試：建立測試環境
print("=== 建立測試環境 ===")
test_dir = Path('search_test')
test_dir.mkdir(exist_ok=True)

# 建立各種檔案
(test_dir / 'small.txt').write_text('Hello' * 10)
(test_dir / 'medium.txt').write_text('World' * 100)
(test_dir / 'data.csv').write_text('a,b,c\n' * 50)
(test_dir / 'script.py').write_text('print("test")' * 20)

# 建立子目錄與檔案
subdir = test_dir / 'subdir'
subdir.mkdir(exist_ok=True)
(subdir / 'nested.txt').write_text('Nested content' * 30)
(subdir / 'config.json').write_text('{"key": "value"}' * 10)

print("\n=== 測試 1: 搜尋所有 .txt 檔案（依大小排序）===")
search_files(test_dir, '*.txt', sort_by='size')

print("\n\n=== 測試 2: 搜尋所有檔案（依名稱排序）===")
search_files(test_dir, '*', sort_by='name', reverse=False)

# 清理
import shutil
shutil.rmtree(test_dir)
print("\n[測試環境已清理]")

### 📊 執行結果
```
=== 建立測試環境 ===

=== 測試 1: 搜尋所有 .txt 檔案（依大小排序）===
搜尋目錄: D:\...\search_test
搜尋模式: *.txt
================================================================================
找到 4 個檔案：

檔名                                   大小             修改時間 路徑
--------------------------------------------------------------------------------
medium.txt                         500 B    2025-10-08 12:30 medium.txt
nested.txt                         420 B    2025-10-08 12:30 subdir\nested.txt
small.txt                           50 B    2025-10-08 12:30 small.txt

================================================================================
總計: 4 個檔案，總大小: 0.9 KB
```

### 📚 知識點總結
- ✅ 使用 `rglob()` 遞迴搜尋檔案
- ✅ 使用 `stat()` 取得檔案大小與時間
- ✅ 使用 `relative_to()` 顯示相對路徑
- ✅ 格式化檔案大小（B/KB/MB）
- ✅ 使用 lambda 函式排序
- ✅ 字串格式化對齊輸出

---

## 範例 2：目錄整理器

### 📋 問題描述
建立一個目錄整理工具，能夠：
1. 將混亂的檔案依副檔名分類到子資料夾
2. 處理同名檔案衝突（自動重新命名）
3. 支援乾跑模式（預覽而不實際移動）
4. 產生整理報告

**難度**：中級

### 🔍 分析思路
1. 遍歷目錄中的所有檔案
2. 根據副檔名決定目標資料夾
3. 檢查並處理同名檔案
4. 移動檔案並記錄操作

### 💻 逐步實作

In [ ]:
from pathlib import Path
from collections import defaultdict

def organize_directory(directory, dry_run=True, custom_categories=None):
    """
    整理目錄：依副檔名分類檔案
    
    參數:
        directory: 要整理的目錄
        dry_run: True=預覽模式，False=實際執行
        custom_categories: 自訂分類規則 {類別名稱: [副檔名列表]}
    """
    # 步驟 1: 預設分類規則
    if custom_categories is None:
        categories = {
            '文件': ['.txt', '.doc', '.docx', '.pdf', '.md'],
            '圖片': ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg'],
            '影片': ['.mp4', '.avi', '.mkv', '.mov'],
            '音樂': ['.mp3', '.wav', '.flac', '.m4a'],
            '程式': ['.py', '.js', '.java', '.cpp', '.c', '.html', '.css'],
            '資料': ['.csv', '.json', '.xml', '.xlsx', '.db'],
            '壓縮': ['.zip', '.rar', '.7z', '.tar', '.gz'],
        }
    else:
        categories = custom_categories
    
    # 步驟 2: 建立副檔名到類別的對應
    ext_to_category = {}
    for category, extensions in categories.items():
        for ext in extensions:
            ext_to_category[ext.lower()] = category
    
    # 步驟 3: 驗證目錄
    source_dir = Path(directory)
    if not source_dir.exists() or not source_dir.is_dir():
        print(f"錯誤：{directory} 不是有效的目錄")
        return
    
    mode = "預覽模式" if dry_run else "執行模式"
    print(f"目錄整理工具 - {mode}")
    print(f"目標目錄: {source_dir.resolve()}")
    print("=" * 70)
    
    # 步驟 4: 掃描檔案
    files_to_organize = []
    for file in source_dir.glob('*'):
        # 只處理檔案，忽略目錄
        if file.is_file():
            files_to_organize.append(file)
    
    if not files_to_organize:
        print("目錄中沒有檔案需要整理")
        return
    
    # 步驟 5: 整理檔案
    operations = defaultdict(list)  # 記錄每個類別的操作
    conflicts = []  # 記錄衝突
    
    for file in files_to_organize:
        # 決定類別
        ext = file.suffix.lower()
        category = ext_to_category.get(ext, '其他')
        
        # 目標目錄
        target_dir = source_dir / category
        target_path = target_dir / file.name
        
        # 處理同名檔案
        if target_path.exists():
            # 產生新檔名: file_1.txt, file_2.txt...
            stem = file.stem
            suffix = file.suffix
            counter = 1
            while target_path.exists():
                target_path = target_dir / f"{stem}_{counter}{suffix}"
                counter += 1
            conflicts.append((file.name, target_path.name))
        
        # 記錄操作
        operations[category].append({
            'source': file,
            'target': target_path,
            'target_dir': target_dir
        })
    
    # 步驟 6: 執行或預覽
    print(f"\n發現 {len(files_to_organize)} 個檔案需要整理：\n")
    
    total_moved = 0
    for category in sorted(operations.keys()):
        ops = operations[category]
        print(f"[{category}] {len(ops)} 個檔案")
        
        # 建立目標目錄
        if not dry_run:
            ops[0]['target_dir'].mkdir(exist_ok=True)
        
        # 移動檔案
        for op in ops:
            print(f"  {op['source'].name} -> {category}/{op['target'].name}")
            if not dry_run:
                op['source'].rename(op['target'])
            total_moved += 1
        print()
    
    # 步驟 7: 報告
    print("=" * 70)
    print(f"整理完成！")
    print(f"  處理檔案: {total_moved} 個")
    print(f"  建立類別: {len(operations)} 個")
    if conflicts:
        print(f"  檔名衝突: {len(conflicts)} 個（已自動重新命名）")
        for original, renamed in conflicts[:3]:  # 只顯示前 3 個
            print(f"    {original} -> {renamed}")
    if dry_run:
        print("\n※ 這是預覽模式，檔案尚未實際移動")
        print("  若要實際執行，請設定 dry_run=False")

# 測試
print("=== 建立測試環境 ===")
test_dir = Path('organize_test')
test_dir.mkdir(exist_ok=True)

# 建立混亂的檔案
test_files = [
    'report.txt', 'presentation.pdf', 'notes.md',
    'photo1.jpg', 'photo2.png', 'diagram.svg',
    'song.mp3', 'podcast.wav',
    'script.py', 'index.html', 'style.css',
    'data.csv', 'config.json',
    'archive.zip'
]

for filename in test_files:
    (test_dir / filename).write_text('test content')

print(f"已建立 {len(test_files)} 個測試檔案\n")

# 測試 1: 預覽模式
print("\n=== 測試 1: 預覽模式 ===")
organize_directory(test_dir, dry_run=True)

# 測試 2: 實際執行
print("\n\n=== 測試 2: 實際執行 ===")
organize_directory(test_dir, dry_run=False)

print("\n=== 整理後的目錄結構 ===")
for item in sorted(test_dir.rglob('*')):
    if item.is_file():
        print(f"  {item.relative_to(test_dir)}")

# 清理
import shutil
shutil.rmtree(test_dir)
print("\n[測試環境已清理]")

### 📊 執行結果
```
=== 測試 2: 實際執行 ===
目錄整理工具 - 執行模式
目標目錄: D:\...\organize_test
======================================================================

發現 14 個檔案需要整理：

[文件] 3 個檔案
  report.txt -> 文件/report.txt
  presentation.pdf -> 文件/presentation.pdf
  notes.md -> 文件/notes.md

[圖片] 3 個檔案
  photo1.jpg -> 圖片/photo1.jpg
  photo2.png -> 圖片/photo2.png
  diagram.svg -> 圖片/diagram.svg

[音樂] 2 個檔案
  song.mp3 -> 音樂/song.mp3
  podcast.wav -> 音樂/podcast.wav

======================================================================
整理完成！
  處理檔案: 14 個
  建立類別: 6 個
  檔名衝突: 0 個
```

### 📚 知識點總結
- ✅ 使用 `glob('*')` 遍歷目錄（不遞迴）
- ✅ 動態建立分類目錄 `mkdir(exist_ok=True)`
- ✅ 處理檔名衝突（自動編號）
- ✅ 使用 `defaultdict` 分組資料
- ✅ 實作乾跑模式（預覽功能）
- ✅ 產生操作報告

---

## 範例 3：自動備份系統

### 📋 問題描述
建立一個簡易備份系統，能夠：
1. 將指定目錄備份到目標位置
2. 只複製新增或修改的檔案（增量備份）
3. 保持目錄結構
4. 產生備份報告

**難度**：中級-進階

### 🔍 分析思路
1. 遍歷來源目錄的所有檔案
2. 比較來源與備份檔案的修改時間
3. 只複製需要更新的檔案
4. 建立必要的目錄結構

### 💻 逐步實作

In [ ]:
from pathlib import Path
from datetime import datetime
import shutil

def backup_directory(source, destination, incremental=True):
    """
    備份目錄
    
    參數:
        source: 來源目錄
        destination: 備份目錄
        incremental: True=增量備份，False=完整備份
    """
    # 步驟 1: 驗證來源目錄
    source_path = Path(source)
    if not source_path.exists() or not source_path.is_dir():
        print(f"錯誤：來源目錄 {source} 不存在")
        return
    
    # 步驟 2: 建立備份目錄
    dest_path = Path(destination)
    dest_path.mkdir(parents=True, exist_ok=True)
    
    backup_mode = "增量備份" if incremental else "完整備份"
    print(f"備份系統 - {backup_mode}")
    print(f"來源: {source_path.resolve()}")
    print(f"目標: {dest_path.resolve()}")
    print(f"開始時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 70)
    
    # 步驟 3: 掃描來源檔案
    all_files = list(source_path.rglob('*'))
    source_files = [f for f in all_files if f.is_file()]
    
    print(f"\n掃描到 {len(source_files)} 個檔案\n")
    
    # 步驟 4: 決定需要備份的檔案
    files_to_backup = []
    skipped_files = []
    
    for source_file in source_files:
        # 計算目標路徑（保持相對結構）
        relative_path = source_file.relative_to(source_path)
        dest_file = dest_path / relative_path
        
        # 決定是否需要複製
        need_copy = False
        reason = ""
        
        if not dest_file.exists():
            need_copy = True
            reason = "新檔案"
        elif not incremental:
            need_copy = True
            reason = "完整備份"
        else:
            # 比較修改時間
            source_mtime = source_file.stat().st_mtime
            dest_mtime = dest_file.stat().st_mtime
            if source_mtime > dest_mtime:
                need_copy = True
                reason = "已修改"
        
        if need_copy:
            files_to_backup.append({
                'source': source_file,
                'dest': dest_file,
                'relative': relative_path,
                'reason': reason
            })
        else:
            skipped_files.append(relative_path)
    
    # 步驟 5: 執行備份
    print(f"需要備份: {len(files_to_backup)} 個檔案")
    print(f"略過: {len(skipped_files)} 個檔案（未修改）\n")
    
    copied_count = 0
    total_size = 0
    
    for item in files_to_backup:
        try:
            # 建立目標目錄
            item['dest'].parent.mkdir(parents=True, exist_ok=True)
            
            # 複製檔案
            shutil.copy2(item['source'], item['dest'])  # copy2 保留時間戳記
            
            # 統計
            size = item['source'].stat().st_size
            total_size += size
            copied_count += 1
            
            # 顯示進度
            size_kb = size / 1024
            print(f"[{item['reason']}] {item['relative']} ({size_kb:.1f} KB)")
            
        except Exception as e:
            print(f"[錯誤] {item['relative']}: {e}")
    
    # 步驟 6: 報告
    print("\n" + "=" * 70)
    print(f"備份完成！")
    print(f"  複製檔案: {copied_count} 個")
    print(f"  略過檔案: {len(skipped_files)} 個")
    print(f"  總大小: {total_size / 1024:.1f} KB")
    print(f"  結束時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    return {
        'copied': copied_count,
        'skipped': len(skipped_files),
        'total_size': total_size
    }

# 測試
print("=== 建立測試環境 ===")

# 建立來源目錄
source_dir = Path('backup_source')
source_dir.mkdir(exist_ok=True)

(source_dir / 'file1.txt').write_text('Content 1' * 100)
(source_dir / 'file2.txt').write_text('Content 2' * 200)

subdir = source_dir / 'data'
subdir.mkdir(exist_ok=True)
(subdir / 'data1.csv').write_text('a,b,c\n' * 50)
(subdir / 'data2.json').write_text('{"key": "value"}' * 30)

print("已建立來源目錄\n")

# 測試 1: 首次完整備份
print("=== 測試 1: 首次備份 ===")
backup_directory(source_dir, 'backup_dest', incremental=True)

# 修改一個檔案
import time
time.sleep(1)  # 確保時間戳記不同
(source_dir / 'file1.txt').write_text('Modified content' * 150)

# 測試 2: 增量備份
print("\n\n=== 測試 2: 增量備份（只備份修改的檔案）===")
backup_directory(source_dir, 'backup_dest', incremental=True)

# 清理
shutil.rmtree(source_dir)
shutil.rmtree('backup_dest')
print("\n[測試環境已清理]")

### 📊 執行結果
```
=== 測試 2: 增量備份（只備份修改的檔案）===
備份系統 - 增量備份
來源: D:\...\backup_source
目標: D:\...\backup_dest
開始時間: 2025-10-08 12:35:20
======================================================================

掃描到 4 個檔案

需要備份: 1 個檔案
略過: 3 個檔案（未修改）

[已修改] file1.txt (1.5 KB)

======================================================================
備份完成！
  複製檔案: 1 個
  略過檔案: 3 個
  總大小: 1.5 KB
  結束時間: 2025-10-08 12:35:21
```

### 📚 知識點總結
- ✅ 使用 `rglob('*')` 遞迴掃描所有檔案
- ✅ 使用 `relative_to()` 計算相對路徑
- ✅ 比較檔案修改時間 `stat().st_mtime`
- ✅ 使用 `shutil.copy2()` 保留時間戳記
- ✅ 使用 `mkdir(parents=True)` 建立多層目錄
- ✅ 增量備份邏輯實作

---

## 範例 4：批次重新命名工具

### 📋 問題描述
建立批次重新命名工具，能夠：
1. 支援多種命名規則（編號、日期、取代文字）
2. 預覽重新命名結果
3. 處理名稱衝突
4. 支援復原操作

**難度**：進階

### 🔍 分析思路
1. 掃描目錄中的檔案
2. 根據規則產生新檔名
3. 檢查衝突並調整
4. 執行重新命名並記錄

### 💻 逐步實作

In [ ]:
from pathlib import Path
from datetime import datetime
import json

def batch_rename(directory, pattern, mode='number', dry_run=True, start=1):
    """
    批次重新命名檔案
    
    參數:
        directory: 目標目錄
        pattern: 檔名模式（可含 {n}, {name}, {date} 等變數）
        mode: 命名模式 ('number', 'date', 'replace')
        dry_run: 預覽模式
        start: 起始編號（mode='number' 時使用）
    
    模式說明:
        - number: 使用編號，如 file_001.txt
        - date: 使用日期，如 2025-10-08_file.txt
        - replace: 取代文字（pattern 為 '舊文字->新文字'）
    """
    # 步驟 1: 驗證目錄
    target_dir = Path(directory)
    if not target_dir.exists() or not target_dir.is_dir():
        print(f"錯誤：{directory} 不是有效的目錄")
        return
    
    mode_text = "預覽模式" if dry_run else "執行模式"
    print(f"批次重新命名工具 - {mode_text}")
    print(f"目標目錄: {target_dir.resolve()}")
    print(f"命名模式: {mode}")
    print(f"命名規則: {pattern}")
    print("=" * 70)
    
    # 步驟 2: 收集檔案
    files = sorted([f for f in target_dir.glob('*') if f.is_file()])
    
    if not files:
        print("目錄中沒有檔案")
        return
    
    print(f"\n找到 {len(files)} 個檔案\n")
    
    # 步驟 3: 產生新檔名
    rename_plan = []
    counter = start
    
    for file in files:
        # 根據模式產生新檔名
        if mode == 'number':
            # 計算需要的位數（自動補零）
            digits = len(str(len(files) + start - 1))
            new_stem = pattern.format(
                n=str(counter).zfill(digits),
                name=file.stem
            )
            new_name = new_stem + file.suffix
            counter += 1
            
        elif mode == 'date':
            # 使用檔案修改日期
            mtime = file.stat().st_mtime
            date_str = datetime.fromtimestamp(mtime).strftime('%Y%m%d')
            new_stem = pattern.format(
                date=date_str,
                name=file.stem
            )
            new_name = new_stem + file.suffix
            
        elif mode == 'replace':
            # 取代文字
            if '->' not in pattern:
                print(f"錯誤：replace 模式需要 '舊文字->新文字' 格式")
                return
            old_text, new_text = pattern.split('->', 1)
            new_name = file.name.replace(old_text, new_text)
        
        else:
            print(f"錯誤：不支援的模式 {mode}")
            return
        
        new_path = file.parent / new_name
        
        # 檢查是否真的需要重新命名
        if file.name != new_name:
            rename_plan.append({
                'old': file,
                'new': new_path,
                'old_name': file.name,
                'new_name': new_name
            })
    
    if not rename_plan:
        print("所有檔案已符合命名規則，無需重新命名")
        return
    
    # 步驟 4: 檢查衝突
    conflicts = []
    new_names = set()
    
    for item in rename_plan:
        if item['new'].exists() and item['new'] != item['old']:
            conflicts.append(item['new_name'])
        if item['new_name'] in new_names:
            conflicts.append(f"{item['new_name']} (重複)")
        new_names.add(item['new_name'])
    
    # 步驟 5: 顯示預覽
    print("重新命名計劃：\n")
    for i, item in enumerate(rename_plan, 1):
        print(f"{i:3d}. {item['old_name']:30s} -> {item['new_name']}")
    
    # 步驟 6: 警告衝突
    if conflicts:
        print("\n⚠️  警告：發現檔名衝突！")
        for conflict in conflicts:
            print(f"  - {conflict}")
        print("\n請修正命名規則後重試")
        return
    
    # 步驟 7: 執行重新命名
    if not dry_run:
        print("\n開始重新命名...")
        
        # 記錄操作（用於復原）
        log = {
            'timestamp': datetime.now().isoformat(),
            'directory': str(target_dir.resolve()),
            'mode': mode,
            'pattern': pattern,
            'operations': []
        }
        
        success = 0
        for item in rename_plan:
            try:
                item['old'].rename(item['new'])
                log['operations'].append({
                    'old': item['old_name'],
                    'new': item['new_name']
                })
                success += 1
            except Exception as e:
                print(f"  錯誤: {item['old_name']} -> {e}")
        
        # 儲存操作記錄
        log_file = target_dir / '.rename_log.json'
        with open(log_file, 'w', encoding='utf-8') as f:
            json.dump(log, f, ensure_ascii=False, indent=2)
        
        print(f"\n完成！成功重新命名 {success} 個檔案")
        print(f"操作記錄已儲存至: {log_file.name}")
    else:
        print("\n※ 這是預覽模式，檔案尚未實際重新命名")
        print("  若要實際執行，請設定 dry_run=False")

# 測試
print("=== 建立測試環境 ===")
test_dir = Path('rename_test')
test_dir.mkdir(exist_ok=True)

# 建立測試檔案
test_files = ['IMG_001.jpg', 'IMG_002.jpg', 'IMG_003.jpg', 
              'document.txt', 'report.txt', 'notes.txt']
for filename in test_files:
    (test_dir / filename).write_text('test')

print(f"已建立 {len(test_files)} 個測試檔案\n")

# 測試 1: 編號模式（預覽）
print("=== 測試 1: 編號模式（預覽）===")
batch_rename(test_dir, 'photo_{n}', mode='number', dry_run=True)

# 測試 2: 取代文字模式（實際執行）
print("\n\n=== 測試 2: 取代文字（實際執行）===")
batch_rename(test_dir, 'IMG_->photo_', mode='replace', dry_run=False)

print("\n重新命名後的檔案：")
for file in sorted(test_dir.glob('*')):
    if file.is_file() and not file.name.startswith('.'):
        print(f"  {file.name}")

# 清理
import shutil
shutil.rmtree(test_dir)
print("\n[測試環境已清理]")

### 📊 執行結果
```
=== 測試 2: 取代文字（實際執行）===
批次重新命名工具 - 執行模式
目標目錄: D:\...\rename_test
命名模式: replace
命名規則: IMG_->photo_
======================================================================

找到 6 個檔案

重新命名計劃：

  1. IMG_001.jpg                     -> photo_001.jpg
  2. IMG_002.jpg                     -> photo_002.jpg
  3. IMG_003.jpg                     -> photo_003.jpg

開始重新命名...

完成！成功重新命名 3 個檔案
操作記錄已儲存至: .rename_log.json
```

### 📚 知識點總結
- ✅ 使用 `rename()` 重新命名檔案
- ✅ 字串格式化產生新檔名 `format()`
- ✅ 檢測檔名衝突
- ✅ 使用 `zfill()` 補零
- ✅ 操作記錄儲存（JSON）
- ✅ 實作預覽與執行模式

---

## 範例 5：檔案大小分析器

### 📋 問題描述
建立目錄分析工具，能夠：
1. 統計目錄中各類型檔案的數量與大小
2. 找出最大的檔案與資料夾
3. 產生視覺化報告
4. 計算目錄樹的深度

**難度**：進階

### 🔍 分析思路
1. 遞迴遍歷所有檔案與目錄
2. 依副檔名分類統計
3. 計算每個目錄的總大小
4. 排序並產生報告

### 💻 逐步實作

In [ ]:
from pathlib import Path
from collections import defaultdict

def analyze_directory(directory, top_n=10):
    """
    分析目錄結構與檔案大小
    
    參數:
        directory: 要分析的目錄
        top_n: 顯示前 N 大的檔案/目錄
    """
    # 步驟 1: 驗證目錄
    root = Path(directory)
    if not root.exists() or not root.is_dir():
        print(f"錯誤：{directory} 不是有效的目錄")
        return
    
    print(f"目錄分析工具")
    print(f"分析目錄: {root.resolve()}")
    print("=" * 70)
    
    # 步驟 2: 初始化統計資料
    stats = {
        'total_files': 0,
        'total_dirs': 0,
        'total_size': 0,
        'by_extension': defaultdict(lambda: {'count': 0, 'size': 0}),
        'largest_files': [],
        'dir_sizes': {},
        'max_depth': 0
    }
    
    # 步驟 3: 掃描所有項目
    print("\n正在掃描檔案...")
    
    all_items = list(root.rglob('*'))
    
    for item in all_items:
        # 計算深度
        depth = len(item.relative_to(root).parts)
        stats['max_depth'] = max(stats['max_depth'], depth)
        
        if item.is_file():
            # 檔案統計
            stats['total_files'] += 1
            size = item.stat().st_size
            stats['total_size'] += size
            
            # 按副檔名分類
            ext = item.suffix.lower() or '(無副檔名)'
            stats['by_extension'][ext]['count'] += 1
            stats['by_extension'][ext]['size'] += size
            
            # 記錄大檔案
            stats['largest_files'].append({
                'path': item,
                'size': size,
                'relative': item.relative_to(root)
            })
            
        elif item.is_dir():
            stats['total_dirs'] += 1
    
    # 步驟 4: 計算每個目錄的大小
    print("計算目錄大小...\n")
    
    for dir_path in [d for d in all_items if d.is_dir()]:
        dir_size = sum(
            f.stat().st_size 
            for f in dir_path.rglob('*') 
            if f.is_file()
        )
        stats['dir_sizes'][dir_path] = dir_size
    
    # 步驟 5: 排序
    stats['largest_files'].sort(key=lambda x: x['size'], reverse=True)
    largest_dirs = sorted(
        stats['dir_sizes'].items(), 
        key=lambda x: x[1], 
        reverse=True
    )
    
    # 步驟 6: 產生報告
    def format_size(size):
        """格式化檔案大小"""
        if size < 1024:
            return f"{size} B"
        elif size < 1024 * 1024:
            return f"{size / 1024:.1f} KB"
        elif size < 1024 * 1024 * 1024:
            return f"{size / (1024 * 1024):.1f} MB"
        else:
            return f"{size / (1024 * 1024 * 1024):.1f} GB"
    
    # 總覽
    print("📊 總覽")
    print(f"  總檔案數: {stats['total_files']:,}")
    print(f"  總目錄數: {stats['total_dirs']:,}")
    print(f"  總大小: {format_size(stats['total_size'])}")
    print(f"  目錄深度: {stats['max_depth']} 層")
    print()
    
    # 按副檔名統計
    print("📁 檔案類型統計（前 10 名）")
    ext_sorted = sorted(
        stats['by_extension'].items(),
        key=lambda x: x[1]['size'],
        reverse=True
    )[:10]
    
    print(f"{'副檔名':<15} {'數量':>8} {'總大小':>15} {'百分比':>10}")
    print("-" * 50)
    
    for ext, data in ext_sorted:
        percentage = (data['size'] / stats['total_size'] * 100) if stats['total_size'] > 0 else 0
        print(f"{ext:<15} {data['count']:>8} {format_size(data['size']):>15} {percentage:>9.1f}%")
    print()
    
    # 最大檔案
    print(f"📄 最大的 {min(top_n, len(stats['largest_files']))} 個檔案")
    print(f"{'大小':>12} {'檔案路徑'}")
    print("-" * 70)
    
    for item in stats['largest_files'][:top_n]:
        print(f"{format_size(item['size']):>12} {item['relative']}")
    print()
    
    # 最大目錄
    print(f"📂 最大的 {min(top_n, len(largest_dirs))} 個目錄")
    print(f"{'大小':>12} {'目錄路徑'}")
    print("-" * 70)
    
    for dir_path, size in largest_dirs[:top_n]:
        relative = dir_path.relative_to(root)
        print(f"{format_size(size):>12} {relative if str(relative) != '.' else '(根目錄)'}")
    
    return stats

# 測試
print("=== 建立測試環境 ===")
test_dir = Path('analyze_test')
test_dir.mkdir(exist_ok=True)

# 建立各種大小的檔案
(test_dir / 'small.txt').write_text('x' * 100)
(test_dir / 'medium.txt').write_text('x' * 5000)
(test_dir / 'large.txt').write_text('x' * 50000)

(test_dir / 'data1.csv').write_text('a,b,c\n' * 1000)
(test_dir / 'data2.csv').write_text('a,b,c\n' * 2000)

(test_dir / 'script1.py').write_text('print("hello")' * 100)
(test_dir / 'script2.py').write_text('print("world")' * 200)

# 建立子目錄
subdir1 = test_dir / 'images'
subdir1.mkdir()
(subdir1 / 'photo1.jpg').write_text('x' * 8000)
(subdir1 / 'photo2.jpg').write_text('x' * 12000)

subdir2 = test_dir / 'documents'
subdir2.mkdir()
(subdir2 / 'report.pdf').write_text('x' * 15000)
(subdir2 / 'notes.txt').write_text('x' * 3000)

print("已建立測試目錄結構\n")

# 執行分析
print("=" * 70)
analyze_directory(test_dir, top_n=5)

# 清理
import shutil
shutil.rmtree(test_dir)
print("\n[測試環境已清理]")

### 📊 執行結果
```
目錄分析工具
分析目錄: D:\...\analyze_test
======================================================================

正在掃描檔案...
計算目錄大小...

📊 總覽
  總檔案數: 11
  總目錄數: 2
  總大小: 103.0 KB
  目錄深度: 2 層

📁 檔案類型統計（前 10 名）
副檔名              數量          總大小     百分比
--------------------------------------------------
.txt                   4         56.5 KB      54.8%
.jpg                   2         19.5 KB      19.0%
.pdf                   1         14.6 KB      14.2%
.csv                   2          5.9 KB       5.7%
.py                    2          4.5 KB       4.4%

📄 最大的 5 個檔案
        大小 檔案路徑
----------------------------------------------------------------------
     48.8 KB large.txt
     14.6 KB documents\report.pdf
     11.7 KB images\photo2.jpg
      7.8 KB images\photo1.jpg
      4.9 KB medium.txt

📂 最大的 5 個目錄
        大小 目錄路徑
----------------------------------------------------------------------
    103.0 KB (根目錄)
     19.5 KB images
     17.6 KB documents
```

### 📚 知識點總結
- ✅ 遞迴統計目錄與檔案
- ✅ 使用 `defaultdict` 分類統計
- ✅ 計算目錄深度 `len(parts)`
- ✅ 多維度排序與篩選
- ✅ 格式化輸出表格
- ✅ 百分比計算與顯示

---

## 🎯 總結

本檔案完成了 5 個實戰範例：

1. **檔案搜尋工具** - 遞迴搜尋、屬性查詢、排序
2. **目錄整理器** - 分類整理、衝突處理、預覽模式
3. **自動備份系統** - 增量備份、時間比較、結構保持
4. **批次重新命名** - 多種模式、衝突檢測、操作記錄
5. **檔案大小分析器** - 統計分析、視覺化報告、多維度排序

### 核心技能
- ✅ Path 物件的綜合應用
- ✅ 檔案系統遍歷與操作
- ✅ 錯誤處理與衝突解決
- ✅ 資料統計與分析
- ✅ 實用工具設計

### 下一步
- 完成 `03-practice.ipynb` 課堂練習
- 完成 `04-exercises.ipynb` 課後習題
- 挑戰 `quiz.ipynb` 自我測驗